In [ ]:
%pip install pandas
%pip install openpyxl

Prebacivanje podataka iz xlsx u csv

In [ ]:
import pandas as pd
import re
import os

def add_zupanija(value):
    if value not in ["Republika Hrvatska", "Grad Zagreb"]:
        return f"{value} Županija"
    return value
dataframes = []

# Specify the Excel file path
file_path = 'data/census/popis_2021-stanovnistvo_po_gradovima_opcinama.xlsx'

# Get the directory where the Excel file is located
file_dir = 'data/census/'

# List of descriptive sheet names (mapping between sheet number and name)
file_name = [
    "narodnost.csv", "vjera.csv", "državljanstvo.csv", "materinski_jezik.csv", "vjeroispovjest.csv", 
    "dob.csv", "najčešća_prezimena.csv", "završena_škola.csv", "pohađanje_škole.csv", 
    "broj_živorođene_djece.csv", "bračni_status.csv", "ekonomska_aktivnost.csv", 
    "zaposlenost_prema_zanimanju.csv", "područje_djelatnosti.csv", 
    "položaj_u_zaposlenju.csv", "migracijska_obilježja.csv", 
    "mjesto_rođenja_stanovanja.csv"
]

# Iterate over the sheet numbers (1 to 17) and use the corresponding descriptive name from the mapping
for i in range(1, 18):
    # Specific sheets need different handling
    if i == 8:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[9] = 'Ukupno'
        df.columns.values[10] = 'Bez škole'
        df.columns.values[11] = '1 – 3 razreda osnovne škole'
        df.columns.values[12] = '4 – 7 razreda osnovne škole'
        df.columns.values[13] = 'Osnovna škola'
        df.columns.values[14] = 'Srednja škola'
        df.columns.values[14] = 'Srednja škola'
        df.columns.values[19] = 'Nepoznato'
    elif i == 9:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=8)
        df.columns.values[7] = 'Ukupno'
        df.columns.values[8] = 'Predškolsko obrazovanje'
        df.columns.values[17] = 'svega'
        df.columns.values[18] = 'stručni'
        df.columns.values[19] = 'specijalistički diplomski'
        df.columns.values[20] = 'svega'
        df.columns.values[21] = 'preddiplomski'
        df.columns.values[22] = 'diplomski, integrirani'
        df.columns.values[23] = 'poslijediplomski specijalistički'
        df.columns.values[24] = 'doktorski'  
        df.columns.values[25] = 'Ne pohađa'
        df = df.drop(index=0)
    elif i == 10:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[7] = 'Ukupno'
        df.columns.values[8] = 'Nisu rodile'
        df.columns.values[20] = 'Nepoznato'
    elif i == 11:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[7] = 'Ukupno'
    elif i == 12:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[9] = 'Ukupno'
        df.columns.values[19] = 'Nepoznato'
    elif i == 13 or i == 14:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df = df.drop(df.columns[6], axis=1)
    elif i == 15:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[9] = 'Ukupno'
        df.columns.values[10] = 'Zaposlenici'
        df.columns.values[14] = 'Pomažući članovi obitelji'
        df.columns.values[15] = 'Ostale zaposlene osobe'
        df.columns.values[16] = 'Nezaposleno'
    elif i == 16:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=8)
        df.columns.values[7] = 'Ukupan broj stanovnika' 
        df.columns.values[8] = 'Od rođenja stanuju u istom naselju'
        df.columns.values[20] = 'Nepoznato'
    elif i == 17:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)
        df.columns.values[7] = 'Ukupan broj stanovnika' 
        df.columns.values[22] = 'Nepoznato'
    else:
        df = pd.read_excel(file_path, sheet_name=f'{i}.', skiprows=7)

    # Truncate each column name at the newline and remove excessive whitespace
    df.columns = [re.sub(r'\s+', ' ', str(col).split('\n')[0]).strip() for col in df.columns]

    # Capitalize only the first letter of each column name
    df.columns = df.columns.str.capitalize()
    
    #Removing english columns
    columns_to_remove = ['Religion', 'Age', 'Sex']
    df = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

    # Remove the second, third, and fourth columns
    df = df.drop(df.columns[[1, 2, 3]], axis=1)

    df.iloc[:, 0] = df.iloc[:, 0].apply(add_zupanija)

    # Update the second column where the first column has the specific word and the second column is empty (Zagreb fix)
    df.loc[(df[df.columns[0]] == 'Grad Zagreb') & (df[df.columns[1]].isna()), df.columns[1]] = 'Zagreb'

    # Get the sheet name
    output_filename = f'{file_name[i-1]}'

    # Define the output CSV file path
    output_path = os.path.join(file_dir, output_filename)
    
    dataframes.append(df)
    # Save the DataFrame to a CSV file in the same directory as the Excel file
    df.to_csv(output_path, index=False)
    print(f'Sheet "{file_name[i-1]}" has been saved as "{output_path}".')


Sheet "narodnost.csv" has been saved as "data/census/narodnost.csv".
Sheet "vjera.csv" has been saved as "data/census/vjera.csv".
Sheet "državljanstvo.csv" has been saved as "data/census/državljanstvo.csv".
Sheet "materinski_jezik.csv" has been saved as "data/census/materinski_jezik.csv".
Sheet "vjeroispovjest.csv" has been saved as "data/census/vjeroispovjest.csv".
Sheet "dob.csv" has been saved as "data/census/dob.csv".
Sheet "najčešća_prezimena.csv" has been saved as "data/census/najčešća_prezimena.csv".
Sheet "završena_škola.csv" has been saved as "data/census/završena_škola.csv".
Sheet "pohađanje_škole.csv" has been saved as "data/census/pohađanje_škole.csv".
Sheet "broj_živorođene_djece.csv" has been saved as "data/census/broj_živorođene_djece.csv".
Sheet "bračni_status.csv" has been saved as "data/census/bračni_status.csv".
Sheet "ekonomska_aktivnost.csv" has been saved as "data/census/ekonomska_aktivnost.csv".
Sheet "zaposlenost_prema_zanimanju.csv" has been saved as "data/cens

In [ ]:
for df in dataframes:
    print(df)

In [ ]:
import pandas as pd
import re

# Function to filter out rows based on specific header values and text patterns
def filter_rows(df):
    # List of headers to be excluded
    exclude_headers = ["Republika Hrvatska","Županija", "County of", "Grad/općina"]
    
    # Filter out rows where the first column matches any of the excluded headers
    df = df[~df.iloc[:, 0].isin(exclude_headers)]
    
    # Define a regex pattern to match rows starting with numbers and parentheses
    pattern = re.compile(r'^\d+\)')
    
    # Filter rows where the first column matches the pattern
    filtered_df = df[~df.iloc[:, 0].astype(str).str.match(pattern)]
    
    # Return the filtered DataFrame
    return filtered_df

# Function to format city names
def format_city_names(city_name):

    city_name = city_name.upper()

    if city_name == "NAN":
        city_name = ''
    # Replace en dash (–) with hyphen (-)
    city_name = city_name.replace('–', '-')
    
    # Format 'C1-C2' to 'C1 - C2'
    if city_name == "MURTER-KORNATI":
        city_name = re.sub(r'(?<=\w)-(?=\w)', ' - ', city_name)
    
    # Strip any leading or trailing spaces
    city_name = city_name.strip()
    
    return city_name

# Specify the CSV file path
csv_file_path = 'data/census/vjera.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, header=None)

# Set the correct headers for the DataFrame
df.columns = ['Županija', 'Grad/općina'] + list(df.columns[2:])

# Filter out unwanted rows
df = filter_rows(df)

# Ensure columns exist and collect non-empty values
zupanija_set = set()
grad_opcina_list = []

if len(df.columns) > 4:
    zupanija_set.update(df.iloc[:, 0].dropna().str.upper())  # Županija
    grad_opcina_list = df.iloc[:, 1].dropna().tolist()  # Grad/općina

# Create DataFrames for values
df_zupanija = pd.DataFrame(zupanija_set, columns=['Županija'])
df_grad_opcina = pd.DataFrame(grad_opcina_list, columns=['Grad/općina'])

# Format the city names in the DataFrame
df_grad_opcina['Grad/općina'] = df_grad_opcina['Grad/općina'].apply(format_city_names)

# Print the formatted Grad/općina DataFrame
print("\nFormatted Grad/općina DataFrame:")
print(df_zupanija)
print("\nFormatted Grad/općina DataFrame:")
print(df_grad_opcina)

# Save the DataFrames to CSV files if needed
# df_zupanija.to_csv('zupanija_unique.csv', index=False)
#df_grad_opcina.to_csv('data/census/grad_opcina_unique.csv', index=False)


Formatted Grad/općina DataFrame:
                           Županija
0       SISAČKO-MOSLAVAČKA ŽUPANIJA
1        OSJEČKO-BARANJSKA ŽUPANIJA
2     VUKOVARSKO-SRIJEMSKA ŽUPANIJA
3               ZAGREBAČKA ŽUPANIJA
4       KRAPINSKO-ZAGORSKA ŽUPANIJA
5               MEĐIMURSKA ŽUPANIJA
6               KARLOVAČKA ŽUPANIJA
7   BJELOVARSKO-BILOGORSKA ŽUPANIJA
8         ŠIBENSKO-KNINSKA ŽUPANIJA
9            LIČKO-SENJSKA ŽUPANIJA
10                ISTARSKA ŽUPANIJA
11       POŽEŠKO-SLAVONSKA ŽUPANIJA
12  DUBROVAČKO-NERETVANSKA ŽUPANIJA
13                      GRAD ZAGREB
14        BRODSKO-POSAVSKA ŽUPANIJA
15   VIROVITIČKO-PODRAVSKA ŽUPANIJA
16      PRIMORSKO-GORANSKA ŽUPANIJA
17                     NAN ŽUPANIJA
18             VARAŽDINSKA ŽUPANIJA
19    SPLITSKO-DALMATINSKA ŽUPANIJA
20                ZADARSKA ŽUPANIJA
21  KOPRIVNIČKO-KRIŽEVAČKA ŽUPANIJA

Formatted Grad/općina DataFrame:
             Grad/općina
0              DUGO SELO
1            IVANIĆ-GRAD
2           JASTREBARSKO
3   

Oblikovanje svih fileova

In [162]:
for file in file_name:
    file_path = os.path.join(file_dir, file)
    if file.endswith('.csv'):
        df = pd.read_csv(file_path)

        df.iloc[:, 0] = df.iloc[:, 0].astype(str)
        df.iloc[:, 0] = df.iloc[:, 0].apply(format_city_names)

        df.iloc[:, 1] = df.iloc[:, 1].astype(str) 
        df.iloc[:, 1] = df.iloc[:, 1].apply(format_city_names)
        
        # Print the modified DataFrame (optional)
        print(df)
        
        # Optionally, save the modified DataFrame back to a CSV
        df.to_csv(file_path, index=False)

                Županija          Grad/općina   Ukupno  Ukupno, %   Hrvati  \
0     REPUBLIKA HRVATSKA                       3871833        100  3547614   
1    ZAGREBAČKA ŽUPANIJA                        299985        100   289929   
2    ZAGREBAČKA ŽUPANIJA            DUGO SELO    17676        100    16633   
3    ZAGREBAČKA ŽUPANIJA          IVANIĆ-GRAD    12982        100    12562   
4    ZAGREBAČKA ŽUPANIJA         JASTREBARSKO    14562        100    14262   
..                   ...                  ...      ...        ...      ...   
589          GRAD ZAGREB              SESVETE    70800        100    67741   
590          GRAD ZAGREB            STENJEVEC    53862        100    50571   
591          GRAD ZAGREB     TREŠNJEVKA - JUG    65324        100    60928   
592          GRAD ZAGREB  TREŠNJEVKA - SJEVER    52974        100    49155   
593          GRAD ZAGREB                TRNJE    40539        100    36812   

     Hrvati, % Albanci Albanci, % Austrijanci Austrijanci, %  .